c:\Users\shaur\anaconda3\envs\rasa_bot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import spacy
import gradio as gr

from neo4j import GraphDatabase

In [6]:
driver = GraphDatabase.driver('bolt://54.89.241.3:7687', auth=('chattels-ceilings-release'))

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [12]:
def extract_entity(question, entity_type):
    doc = nlp(question)
    for ent in doc.ents:
        if ent.label_ in entity_type:
            return ent.text
    return None

In [11]:
def get_answer(question):
    doc = nlp(question)
    movie_title = extract_entity(question, ["WORK_OF_ART", "MOVIE"])
    director_name = extract_entity(question, ["PERSON"])
    if movie_title is None:
        return "Sorry, I don't understand what movie you're asking about."
    
    with driver.session() as session:
        if "release" in question or "released" in question:
            result = session.run(f"MATCH (m:Movie) WHERE m.title = '{movie_title}' RETURN m.released")
            if result.peek() is None:
                return f"No information found for the movie '{movie_title}'"
            for record in result:
                return f"The release date of '{movie_title}' is: {record['m.released']}"
        else:
            return "Sorry, I don't understand what you're asking."

In [8]:
# def get_answer(question):
#     movie_title = extract_entity(question, ['WORK_OF_ART', 'MOVIE'])
#     director_name = extract_entity(question, ['PERSON'])

#     if movie_title is None:
#         return "Sorry, I don't understand about the movie you are asking"
    
#     with driver.session() as session:
#         if 'release' in question or 'released' in question:
#             result = session.run(f"MATCH (m:Movie) WHERE m.title = '{movie_title}' RETURN m.released")
#             if result.peek() is None:
#                 return f"No information about the movie is found '{movie_title}'"
#             for record in result:
#                 return f"The release date of '{movie_title}' is: {record['m.released']}"
#         else:
#             return "Sorry, I don't understand what you are asking"

In [13]:
def chatbot(input, history=[]):
    output = get_answer(input)
    history.append((input, output))
    return history, history

In [14]:
gr.Interface(fn = chatbot,
             inputs = ["text",'state'],
             outputs = ["chatbot",'state']).launch(debug = True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\shaur\anaconda3\envs\rasa_bot\lib\site-packages\gradio\queueing.py", line 521, in process_events
    response = await route_utils.call_process_api(
  File "c:\Users\shaur\anaconda3\envs\rasa_bot\lib\site-packages\gradio\route_utils.py", line 276, in call_process_api
    output = await app.get_blocks().process_api(
  File "c:\Users\shaur\anaconda3\envs\rasa_bot\lib\site-packages\gradio\blocks.py", line 1945, in process_api
    result = await self.call_function(
  File "c:\Users\shaur\anaconda3\envs\rasa_bot\lib\site-packages\gradio\blocks.py", line 1513, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "c:\Users\shaur\anaconda3\envs\rasa_bot\lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "c:\Users\shaur\anaconda3\envs\rasa_bot\lib\site-packages\anyio\_backends\_asyncio.py", line 2177, in run_sync_in_worker_thread
    return await

Keyboard interruption in main thread... closing server.
